<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Strato75/Covid-19_ItalyStats/blob/master/notebooks/per_province_choropleth_diagram.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" />Run in Google Colab</a>
  </td>
</table>

In [ ]:
# Execute this cell to install these required packages if you run it in Google Colab
!pip install geojson
!pip install geopandas
!pip install folium

In [ ]:
%pylab inline
import pandas as pd
import requests
import numpy as np
import folium
import pandas as pd
import geopandas as gpd
import geojson 
from matplotlib.colors import rgb2hex
import matplotlib.pyplot as plt
from branca.colormap import linear
from folium.plugins import TimeSliderChoropleth
import matplotlib.dates as mdates 

In [ ]:
## Globals
x_tick_fs = 6

## Functions
def get_df_from_json_url(url):
    response = requests.get(url)
    df = pd.read_json(response.text, orient='records')
    return df


def get_geojson_from_url(url):
    response = requests.get(url)
    geoj = geojson.loads(response.text)
    return geoj


def get_choropleth(geoj, gdf_per_date, cen_lat, cen_lon, col_name, name='', legend_name=''):
    # create the map
    f = folium.Figure(width=650, height=850)

    m = folium.Map(location=[cen_lat,cen_lon], 
                   tiles='Stamen Toner', zoom_start=6,
                   control_scale=True,
                   zoom_control=True)

    gdf_per_date[col_name].replace(0.0, np.nan, inplace=True) # It makes blank background to provinces with zero cases 

    ## add chloropleth layer
    m.choropleth(
        geo_data=geoj,
        name='Totale casi per provincia',
        data=gdf_per_date,
        columns=['prov_name', col_name],
        fill_color='YlOrRd',
        key_on='feature.properties.prov_name',
        legend_name='totale casi per 100000 abitanti al %s' % selected_date.date(),
        nan_fill_color = 'w',
        nan_fill_opacity = 0
    )  

    gdf_per_date[col_name].replace(np.nan, 0.0, inplace=True) 

    groups_by_province = gdf_per_date.groupby('prov_name')
    for prov in groups_by_province.groups:
        if 'In fase di definizione' not in prov:
            ser = groups_by_province.get_group(prov)
            lat = float(ser['lat'])
            long = float(ser['long'])
            tot_cases = int(ser[col_name])

            if tot_cases:
                folium.Circle(
                location= [lat, long],
                tooltip='%s: %d casi' % (prov, tot_cases),
                radius=(30.),
                color='crimson',
                fill=True,
                fill_color='crimson'
                ).add_to(m)

    # enable layers to be turned in or out
    folium.LayerControl().add_to(m)

    f.add_child(m)
    
    return f


### Function to create the TimeSliderChoropleth
def get_TimeSliderChoropleth(gdf, gdf_merged, cen_lat, cen_lon, feature='prov_name', color_field='totale_casi'):
    # Convert timestamps to string epochs
    dates_str = [i.strftime("%s") for i in dates]
    styledata = {}
    
    group_by_feat = gdf_merged.groupby(feature)
    
    ### Creation of styledata structure
    for index in gdf.index:
        feat = gdf.loc[index][feature]
        df = pd.DataFrame({'color': group_by_feat.get_group(feat)[color_field].values, 
                           'opacity': 1.0 }, index = dates_str)
        styledata[index] = df

    ## Compute the color dynamic range
    max_color, min_color, max_opacity, min_opacity = 0, 0, 0, 0

    for prov, data in styledata.items():
        max_color = max(max_color, data['color'].max())
        min_color = min(max_color, data['color'].min())
        max_opacity = max(max_color, data['opacity'].max())
        max_opacity = min(max_color, data['opacity'].max())

    ## Convert color data to HEX and create a color bar
    cmap = linear.PuRd_09.scale(min_color, max_color)

    for x, data in styledata.items():
        data['color'] = data['color'].apply(cmap)

    ### Convert styledata to a dict to be used with a TimeSliderChoropleth 
    styledict = {
        str(index): data.to_dict(orient='index') for
        index, data in styledata.items()
    }

    f = folium.Figure(width=650, height=850)

    m = folium.Map(location=[cen_lat,cen_lon], 
                   tiles='Stamen Toner', zoom_start=6, min_zoom=6, max_zoom=6,
                   control_scale=True,
                   zoom_control=True)

    g = TimeSliderChoropleth(
        data=gdf.to_json(),
        styledict=styledict,

    ).add_to(m)

    folium.LayerControl().add_to(m)

    f.add_child(m)
    
    return f

In [ ]:
raw_it_province_df = get_df_from_json_url('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-province.json')
raw_it_province_df['data'] = pd.to_datetime(raw_it_province_df['data'], format='%Y-%m-%d %H:%M:%S')

# Parameters for savgol filter (Smooting curves)
sliding_win_size = 15
polynomial_fit_degree = 3

In [ ]:
### Data preprocessing
## Add a column to dataframe to merge P.A. Bolzano and P.A. Trento as single region to simplify mulitplot diagram
raw_it_province_df['new_den_regione'] = raw_it_province_df['denominazione_regione']
raw_it_province_df['new_den_regione'].replace('P.A. Bolzano', 'Trentino-Alto Adige', inplace=True)
raw_it_province_df['new_den_regione'].replace('P.A. Trento', 'Trentino-Alto Adige', inplace=True)

# Data in Popolazione_province.json taken from http://dati.istat.it/
prov_pop_df = pd.read_json('Popolazione_province.json')
prov_pop_df['Territorio'].replace("Valle d'Aosta / Vallée d'Aoste", "Aosta", inplace=True)
prov_pop_df['Territorio'].replace("Bolzano / Bozen", "Bolzano", inplace=True)
prov_pop_df['Territorio'].replace("Massa-Carrara", "Massa Carrara", inplace=True)

prov_pop_df.drop_duplicates(subset ="Territorio", keep = 'first', inplace = True)

def population_from_df(prov, df):
    row = df[df['Territorio'] == prov]
    if not row.empty:
        return int(row['Value'])
    else:
        return -1
    
## Add population and total normalized cases column to province dataframe
norm_factor = 100000 # to get cases per 100000 inhabitants
raw_it_province_df['popolazione_provincia'] = raw_it_province_df['denominazione_provincia'].apply(lambda x: population_from_df(x, prov_pop_df))
raw_it_province_df['totale_casi_norm'] = (norm_factor * raw_it_province_df['totale_casi'] / raw_it_province_df['popolazione_provincia']).astype(int)

### Integrating geojson data
geojson_province_url = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson'
geoj = get_geojson_from_url(geojson_province_url)
gdf = gpd.GeoDataFrame.from_features(geoj)
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y

cen_lat = np.median(gdf['centroid_lat'].values)
cen_lon = np.median(gdf['centroid_lon'].values)

## Add to geodataframe latitude, longitute and total case fields
# merge with some columns of raw_it_province_df data 
raw_it_province_df_small = raw_it_province_df[['data', 'codice_provincia', 'lat', 'long', 'totale_casi', 'totale_casi_norm', 'popolazione_provincia']]
gdf_province = gdf.merge(raw_it_province_df_small, left_on='prov_istat_code_num', right_on='codice_provincia')
group_by_date = gdf_province.groupby('data')
group_by_prov = gdf_province.groupby('prov_name')
dates = list(group_by_date.groups)

### Add two new columns to simplify the creation of a TimeSliderChoropleth
gdf_province['color'] = gdf_province['totale_casi']
gdf_province['opacity'] = 1.0

### Create a geopandas dataframe with strings for data in order to be serialized to json
gdf_timeslide = gdf_province.copy(deep=True)
gdf_timeslide["data"] = gdf_timeslide["data"].apply(lambda x: x.strftime("%Y-%m-%d"))

### Total case trends per province for each region

In [ ]:
group_by_region = raw_it_province_df.groupby('new_den_regione')

fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(25,16))
for index, g in enumerate(group_by_region.groups):
        curr_ax = ax[index//5][index%5]
        reg_df = group_by_region.get_group(g)
        g_by_province = reg_df.groupby('denominazione_provincia')

        for p in g_by_province.groups:
            if 'fase di' not in p:
                prov_df = g_by_province.get_group(p)
                #x_data = ['-'.join(i.split('T')[0].split('-')[1:]) for i in np.datetime_as_string(prov_df['data'].values)]
                x_data = prov_df['data'].values
                y_data = prov_df['totale_casi']
                curr_ax.plot(x_data, y_data, 'o-', ms=4, label='%s' % p)

        curr_ax.legend()
        curr_ax.grid()
        curr_ax.set_title(g)
        _ = curr_ax.set_xticklabels(x_data, rotation=90, fontsize=x_tick_fs)
        _ = curr_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) 
        _ = curr_ax.xaxis.set_major_locator(plt.MaxNLocator(40))
        _ = plt.setp(curr_ax.get_xticklabels(), rotation = 90, fontsize=x_tick_fs)
        
        if index%5 == 0:
            curr_ax.set_ylabel('# casi')
plt.tight_layout(rect=[0, 0.01, 1, 0.95])
plt.suptitle("Andamento dei casi totali per provincia", fontsize=18)
fig.savefig('figures/provinces_trend.png', dpi=300, transparency=False, )

### Choropleth Map

#### Get data from the last day

In [ ]:
# Select by date
selected_date = dates[-1] # This select the last day available in the dataframe
gdf_per_date = group_by_date.get_group(selected_date) # Get dataframe by date

### Choropleth with total cases
fc = get_choropleth(geoj, gdf_per_date, cen_lat, cen_lon, col_name = 'totale_casi', 
                         name='Totale casi per provincia', 
                         legend_name='totale casi al %s' % selected_date.date())

### Choropleth with normalized total cases (# cases per 100000 inhabitants)
fc_norm = get_choropleth(geoj, gdf_per_date, cen_lat, cen_lon, col_name = 'totale_casi_norm', 
                         name='Totale casi per provincia', 
                         legend_name='totale casi per 100000 abitanti al %s' % selected_date.date())

In [ ]:
fc.save('figures/province_choropleth_map.html') 
fc_norm.save('figures/province_choropleth_map_normalized.html') 

#### Hover above red points to get the number of cases of that province

In [ ]:
fc

### Timeslider choropleth

In [ ]:
f_tot = get_TimeSliderChoropleth(gdf, gdf_province, cen_lat, cen_lon, feature='prov_name', color_field='totale_casi')
f_tot_norm = get_TimeSliderChoropleth(gdf, gdf_province, cen_lat, cen_lon, feature='prov_name', color_field='totale_casi_norm')

In [ ]:
f_tot.save("figures/timesliderchoroplet_prov_totcasi.html")
f_tot_norm.save("figures/timesliderchoroplet_prov_totcasi_norm.html")

In [ ]:
f_tot